In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 200
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectSimulatorCollection()
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 0.3))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 0.3, 1., 0.2))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=1.))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
wls_analyzer = WLSAnalyzer("WLS", alpha)
analyzers.add_analyzer(wls_analyzer)
analyzers.add_analyzer(MetaAnalyzer(wls_analyzer, "Meta-WLS", alpha))

ipw_model_analyzer = IPWAnalyzer("IPW-Model", alpha, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, "Meta-IPW-Model", alpha))
ipw_robust_analyzer = IPWAnalyzer("IPW-Robust", alpha)
analyzers.add_analyzer(ipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_robust_analyzer, "Meta-IPW-Robust", alpha))

sipw_robust_analyzer = SIPWAnalyzer("SIPW-Model", alpha, robust=False)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, "Meta-SIPW-Model", alpha))
sipw_robust_analyzer = SIPWAnalyzer("SIPW-Robust", alpha)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, "Meta-SIPW-Robust", alpha))

ipw_model_analyzer = IPWAnalyzer("IPW-DR-Model", alpha, dr=True, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, "Meta-IPW-DR-Model", alpha))
ipw_dr_analyzer = IPWAnalyzer("IPW-DR-Robust", alpha, dr=True)
analyzers.add_analyzer(ipw_dr_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_dr_analyzer, "Meta-IPW-DR-Robust", alpha))

sipw_dr_model_analyzer = SIPWAnalyzer("SIPW-DR-Model", alpha, dr=True, robust=False)
analyzers.add_analyzer(sipw_dr_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_dr_model_analyzer, "Meta-SIPW-DR-Model", alpha))
sipw_dr_robust_analyzer = SIPWAnalyzer("SIPW-DR-Robust", alpha, dr=True)
analyzers.add_analyzer(sipw_dr_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_dr_robust_analyzer, "Meta-SIPW-DR-Robust", alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_a()
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.77it/s]


In [3]:
digits = 3
results = results.process(digits=digits)
results.user_results_df

WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.168 (0.002)   
                         Shared Effect            0.164 (0.002)   
                         Distinct Effects         0.165 (0.002)   
                         Moderated Effects        0.448 (0.002)   
                         Noisy Moderated Effects  0.468 (0.002)   
            Avg Sq. Bias No Effect                        0.031   
                         Shared Effect                    0.029   
                         Distinct Effects                 0.030   
                         Moderated Effects                0.208   
                         Noisy Moderated Effects          0.225   
            MSE          No Effect                0.053 (0.001)   
                         Shared Effect            0.051 (0.001)   
                         Distinct Effects         0.051 (0.001)   
                         Moderated Effects        0.240 (0.002)   
                         Noisy Moderated Effects  0.263 (0.003)   
            Coverage     No Effect                0.792 (0.006)   
                         Shared Effect            0.803 (0.005)   
                         Distinct Effects         0.810 (0.006)   
                         Moderated Effects        0.386 (0.006)   
                         Noisy Moderated Effects  0.451 (0.006)   
            CI Length    No Effect                0.597 (0.000)   
                         Shared Effect            0.597 (0.000)   
                         Distinct Effects         0.597 (0.000)   
                         Moderated Effects        0.786 (0.000)   
                         Noisy Moderated Effects  0.888 (0.001)   

                                                       Meta-WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.168 (0.002)   
                         Shared Effect            0.164 (0.002)   
                         Distinct Effects         0.166 (0.002)   
                         Moderated Effects        0.440 (0.002)   
                         Noisy Moderated Effects  0.461 (0.002)   
            Avg Sq. Bias No Effect                        0.030   
                         Shared Effect                    0.028   
                         Distinct Effects                 0.032   
                         Moderated Effects                0.200   
                         Noisy Moderated Effects          0.221   
            MSE          No Effect                0.044 (0.001)   
                         Shared Effect            0.043 (0.001)   
                         Distinct Effects         0.047 (0.001)   
                         Moderated Effects        0.222 (0.002)   
                         Noisy Moderated Effects  0.242 (0.002)   
            Coverage     No Effect                0.783 (0.006)   
                         Shared Effect            0.791 (0.006)   
                         Distinct Effects         0.775 (0.006)   
                         Moderated Effects        0.302 (0.006)   
                         Noisy Moderated Effects  0.293 (0.006)   
            CI Length    No Effect                0.537 (0.000)   
                         Shared Effect            0.537 (0.000)   
                         Distinct Effects         0.538 (0.000)   
                         Moderated Effects        0.707 (0.001)   
                         Noisy Moderated Effects  0.739 (0.002)   

                                                      IPW-Model  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.004 (0.005)   
                         Shared Effect            0.002 (0.003)   
                         Distinct Effects         0.003 (0.004)   
                         Moderated Effects        0.000 (0.004)   
                         Noisy Moderat

$\tau_i \sim \text{Normal}(\mu, \sigma^2)$

$\hat{\tau}_i \sim \text{Normal}(\tau_i, \omega_i^2)$